In [6]:
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from Bio.PDB import PDBParser, Superimposer

In [4]:
def compute_average_plddt(pdb_file):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)

    b_factors = []

    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    b_factors.append(atom.bfactor)

    if b_factors:
        average_bfactor = sum(b_factors) / len(b_factors)
        return average_bfactor
    else:
        return None

def get_ca_atoms(pdb_file):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('structure', pdb_file)
    ca_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if 'CA' in residue:
                    ca_atoms.append(residue['CA'])
    return ca_atoms

def compute_rmsd(file1, file2):
    ca_atoms1 = get_ca_atoms(file1)
    ca_atoms2 = get_ca_atoms(file2)

    if len(ca_atoms1) != len(ca_atoms2):
        raise ValueError("The number of C-alpha atoms in the two structures are different.")

    # Extract the coordinates
    coords1 = np.array([atom.get_coord() for atom in ca_atoms1])
    coords2 = np.array([atom.get_coord() for atom in ca_atoms2])

    # Superimpose the structures
    super_imposer = Superimposer()
    super_imposer.set_atoms(ca_atoms1, ca_atoms2)
    super_imposer.apply(ca_atoms2)
    
    # Calculate RMSD
    rmsd = super_imposer.rms
    return rmsd

In [7]:
folder = Path("/nfs/homedirs/hetzell/code/protein_design/example_outputs/25Jul24_dna_design_0.2/openfold_monomer/predictions")
template = Path("/nfs/homedirs/hetzell/code/protein_design/example_outputs/25Jul24_dna_design_0.2/pdbs/fold_tale_happb_model_0.pdb")
pdb_dict = {}

for pdb_file in tqdm(folder.iterdir()):
    if not pdb_file.is_file() or not pdb_file.suffix == ".pdb":
        continue
    average_plddt = compute_average_plddt(pdb_file)
    rmsd = compute_rmsd(template, pdb_file)
    pdb_dict[pdb_file.stem] = [average_plddt, rmsd]
    # print(f"The average pLDDT score is: {average_plddt}")

0it [00:00, ?it/s]

301it [01:35,  3.14it/s]


In [10]:
df = pd.DataFrame.from_dict(pdb_dict, orient="index", columns=["average_plddt", "rmsd"])

# sort df according to average_plddt
df = df.sort_values(by="rmsd", ascending=True)

In [13]:
df[:20]

,average_plddt,rmsd
fold_tale_happb_model_0_100_A__model_1_unrelaxed,83.716574,6.043256
fold_tale_happb_model_0_100_A__model_1_relaxed,83.848792,6.045082
fold_tale_happb_model_0_130_A__model_1_relaxed,86.332327,6.184440
fold_tale_happb_model_0_130_A__model_1_unrelaxed,86.245648,6.185720
fold_tale_happb_model_0_77_A__model_1_unrelaxed,85.643351,8.244129
fold_tale_happb_model_0_77_A__model_1_relaxed,85.771618,8.245649
fold_tale_happb_model_0_81_A__model_1_unrelaxed,84.798087,8.270655
fold_tale_happb_model_0_81_A__model_1_relaxed,84.957787,8.272613
fold_tale_happb_model_0_20_A__model_1_unrelaxed,83.858173,8.294144
fold_tale_happb_model_0_20_A__model_1_relaxed,84.020045,8.298247


In [12]:
df.to_csv("average_plddt.csv")